In [317]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
import pandas as pd
import numpy as np
import requests
import json, os, pprint, time, re
from bs4 import BeautifulSoup as bs
from datetime import date
import holidays
import matplotlib.pyplot as plt
import seaborn as sns

# 讀取檔案

df = pd.read_json('raw_detail_300000_330093.json')

# 'NaN','','0',0,轉成''

for column_name in df.columns:
    df.loc[df[column_name] == 'NaN', column_name] = ''
    df.loc[df[column_name] == '' , column_name] = ''
    df.loc[df[column_name] == '0', column_name] = ''
    df.loc[df[column_name] == 0 , column_name] = ''
    
# 'title' , 'en_title' 篩掉標點符號 留英文 並將結果合併至 'title_sum_reg_new'

df['title_reg_new'] = df['title'].str.replace('[^a-zA-Z0-9 ]',' ',regex = True).str.replace(' +',' ',regex = True).str.extract('([ \dA-Za-z]*)')
df['title_sum_reg_new'] = df['en_title'].str.replace('[^a-zA-Z0-9 ]',' ',regex = True).str.replace(' +',' ',regex = True).str.extract('([ \dA-Za-z]*)')
df.loc[df['title_reg_new'] == ' ','title_reg_new'] = ''
df.loc[df['title_sum_reg_new'] == ' ','title_sum_reg_new'] = ''
df.loc[df['title_sum_reg_new'] == '','title_sum_reg_new'] = df.loc[df['title_sum_reg_new'] == '','title_reg_new']

# 如果是欄位是list 判定 是否有空[] 結果放進新的直欄

for column_name in df.columns:
    df.loc[df[column_name] == 'NaN', column_name] = ''
    df.loc[df[column_name] == '' , column_name] = ''
    df.loc[df[column_name] == '0', column_name] = ''
    df.loc[df[column_name] == 0 , column_name] = ''
    list_index = []
    if type(df[column_name][0]) == type(list_index) :
        df[f"{column_name}_is_emptylist_new"] = False
        for index , element in enumerate(df[column_name]) :
            if len(element) == 0:
                list_index.append(index)
        if len(list_index) > 0:
            df.loc[df.index.isin(list_index),f"{column_name}_is_emptylist_new"] = True
            
# 暫定 => 新增假日欄位 , 預設 False

us_holidays = holidays.country_holidays('US')
us_holidays['2023-1-1']
for holidayname in us_holidays.values() :
    df[holidayname] = False
    
    
# 判定發行商是否是美國 結果放進新的直欄

list_index = []
for index , countrylist in enumerate(df['production_countries']) :
    df['country_is_USA_new'] = False
    if 'United States of America' in countrylist :
        list_index.append(index)
if len(list_index) > 0:
    df.loc[df.index.isin(list_index),"country_is_USA_new"] = True

# movie_id 去重

df = df[df.duplicated(subset='movie_id') == False]

df.to_json('movieresult.json')

In [308]:
df = pd.read_json('raw_detail_300000_330093.json')

In [309]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26748 entries, 0 to 26747
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_id              26748 non-null  int64  
 1   imdb_id               26748 non-null  object 
 2   title                 26748 non-null  object 
 3   en_title              26748 non-null  object 
 4   budget                26748 non-null  int64  
 5   revenue               26748 non-null  int64  
 6   original_language     26748 non-null  object 
 7   release_date          26748 non-null  object 
 8   avg_rating            26748 non-null  float64
 9   vote_count            26748 non-null  int64  
 10  sequal                26748 non-null  object 
 11  casts                 26748 non-null  object 
 12  keywords              26748 non-null  object 
 13  genres                26748 non-null  object 
 14  production_companies  26748 non-null  object 
 15  adult              

In [310]:
# 'NaN','','0',0,轉成''
for column_name in df.columns:
    df.loc[df[column_name] == 'NaN', column_name] = ''
    df.loc[df[column_name] == '' , column_name] = ''
    df.loc[df[column_name] == '0', column_name] = ''
    df.loc[df[column_name] == 0 , column_name] = ''

In [311]:
# 'title' , 'en_title' 篩掉標點符號 留英文 並將結果合併至 'title_sum_reg_new'
df['title_reg_new'] = df['title'].str.replace('[^a-zA-Z0-9 ]',' ',regex = True).str.replace(' +',' ',regex = True).str.extract('([ \dA-Za-z]*)')
df['title_sum_reg_new'] = df['en_title'].str.replace('[^a-zA-Z0-9 ]',' ',regex = True).str.replace(' +',' ',regex = True).str.extract('([ \dA-Za-z]*)')
df.loc[df['title_reg_new'] == ' ','title_reg_new'] = ''
df.loc[df['title_sum_reg_new'] == ' ','title_sum_reg_new'] = ''
df.loc[df['title_sum_reg_new'] == '','title_sum_reg_new'] = df.loc[df['title_sum_reg_new'] == '','title_reg_new'] 

In [312]:
# 如果是欄位是list 判定 是否有空[] 結果放進新的直欄
for column_name in df.columns:
    df.loc[df[column_name] == 'NaN', column_name] = ''
    df.loc[df[column_name] == '' , column_name] = ''
    df.loc[df[column_name] == '0', column_name] = ''
    df.loc[df[column_name] == 0 , column_name] = ''
    list_index = []
    if type(df[column_name][0]) == type(list_index) :
        df[f"{column_name}_is_emptylist_new"] = False
        for index , element in enumerate(df[column_name]) :
            if len(element) == 0:
                list_index.append(index)
        if len(list_index) > 0:
            df.loc[df.index.isin(list_index),f"{column_name}_is_emptylist_new"] = True

In [313]:
# 暫定 => 新增假日欄位 , 預設 False
us_holidays = holidays.country_holidays('US')
us_holidays['2023-1-1']
for holidayname in us_holidays.values() :
    df[holidayname] = False

New Year's Day
New Year's Day (Observed)
Martin Luther King Jr. Day
Washington's Birthday
Memorial Day
Juneteenth National Independence Day
Independence Day
Labor Day
Columbus Day
Veterans Day
Veterans Day (Observed)
Thanksgiving
Christmas Day


In [314]:
# 判定發行商是否是美國
list_index = []
for index , countrylist in enumerate(df['production_countries']) :
    df['country_is_USA_new'] = False
    if 'United States of America' in countrylist :
        list_index.append(index)
if len(list_index) > 0:
    df.loc[df.index.isin(list_index),"country_is_USA_new"] = True

In [315]:
# movie_id 去重
df = df[df.duplicated(subset='movie_id') == False]

In [316]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26748 entries, 0 to 26747
Data columns (total 43 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   movie_id                               26748 non-null  object
 1   imdb_id                                26748 non-null  object
 2   title                                  26748 non-null  object
 3   en_title                               26748 non-null  object
 4   budget                                 26748 non-null  object
 5   revenue                                26748 non-null  object
 6   original_language                      26748 non-null  object
 7   release_date                           26748 non-null  object
 8   avg_rating                             26748 non-null  object
 9   vote_count                             26748 non-null  object
 10  sequal                                 26748 non-null  object
 11  casts          